<a href="https://colab.research.google.com/github/amankhullar/cs20/blob/master/Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center> <h1> CS20 Assignment 1 </h1> </center>
## Problem 1



In [0]:
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVL'] = '2'

import tensorflow as tf

In [2]:
sess = tf.InteractiveSession()

x = tf.random_uniform([])
y = tf.random_uniform([])

op = tf.cond(x > y, lambda : x + y, lambda : x - y)

print(sess.run(op))

1.1196562


In [3]:
x = tf.random_uniform(shape = [], minval = -1, maxval = 1, name = "x")
y = tf.random_uniform(shape = [], minval = -1, maxval = 1, name = "y")

# Using a tf.Tensor as a Python 'bool' is not allowed
# out = tf.zeros(shape = [1])

# if tf.less(x, y):
#     out = x + y
# elif tf.greater(x, y):
#     out = x - y
# else:
#     out = 0
    
def f1():
    return x + y

def f2():
    return x - y

def f3():
    return tf.zeros(shape = [1])

out = tf.case({tf.less(x, y) : f1, tf.greater(x, y) : f2}, default = f3, name = 'output', exclusive = True)
print(sess.run(out))

0.27322483
